In [1]:
!pip install pydantic-ai


  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 11.0 MB/s eta 0:00:00
Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
  Attempting uninstall: groq
    Found existing installation: groq 0.11.0
    Uninstalling groq-0.11.0:
      Successfully uninstalled groq-0.11.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.28.1 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
unstructured-client 0.28.1 requires pydantic<2.10.0,>=2.9.2, but you have pydantic 2.10.4 which is incompatible.


In [21]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [22]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o', system_prompt='Be a helpful assistant.')



In [23]:
# import nest_asyncio
# nest_asyncio.apply()

In [24]:
# result = agent.run_sync('Tell me a joke.')
result = agent.run_sync('Tell me a joke.')

In [25]:
result


RunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, 120023, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", part_kind='text')], timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, tzinfo=datetime.timezone.utc), kind='response')], _new_message_index=0, data="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", _result_tool_name=None, _usage=Usage(requests=1, request_tokens=21, response_tokens=18, total_tokens=39, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0}))

In [31]:
#> Did you hear about the toothpaste scandal? They called it Colgate.

# all messages from the run
print(result.new_messages())

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, 120023, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[TextPart(content="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", part_kind='text')], timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, tzinfo=datetime.timezone.utc), kind='response')]


In [34]:
result.all_messages()


[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.', part_kind='system-prompt'), UserPromptPart(content='Tell me a joke.', timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, 120023, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content="Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!", part_kind='text')], timestamp=datetime.datetime(2025, 1, 7, 7, 30, 25, tzinfo=datetime.timezone.utc), kind='response')]

In [33]:
result.data


"Sure! Here's one: Why don't scientists trust atoms? Because they make up everything!"

## roulette_wheel

In [36]:
from pydantic_ai import Agent, RunContext

roulette_agent = Agent(  
    'openai:gpt-4o',
    deps_type=int,
    result_type=bool,
    system_prompt=(
        'Use the `roulette_wheel` function to see if the '
        'customer has won based on the number they provide.'
    ),
)


@roulette_agent.tool
async def roulette_wheel(ctx: RunContext[int], square: int) -> str:  # ctx is the context of the run
    """check if the square is a winner"""
    return 'winner' if square == ctx.deps else 'loser'


# Run the agent
success_number = 18  
result = roulette_agent.run_sync('Put my money on square eighteen', deps=success_number)
print(result.data)  
#> True

result = roulette_agent.run_sync('I bet five is the winner', deps=success_number)
print(result.data)
#> False

True
False


## run_agent

In [37]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o')

result_sync = agent.run_sync('What is the capital of Italy?')
print(result_sync.data)
#> Rome


async def main():
    result = await agent.run('What is the capital of France?')
    print(result.data)
    #> Paris

    async with agent.run_stream('What is the capital of the UK?') as response:
        print(await response.get_data())
        #> London

The capital of Italy is Rome.


In [38]:
from pydantic_ai import Agent
from pydantic_ai.exceptions import UsageLimitExceeded
from pydantic_ai.usage import UsageLimits

agent = Agent('openai:gpt-4o')

result_sync = agent.run_sync(
    'What is the capital of Italy? Answer with just the city.',
    usage_limits=UsageLimits(response_tokens_limit=10),
)
print(result_sync.data)
#> Rome
print(result_sync.usage())
"""
Usage(requests=1, request_tokens=62, response_tokens=1, total_tokens=63, details=None)
"""

try:
    result_sync = agent.run_sync(
        'What is the capital of Italy? Answer with a paragraph.',
        usage_limits=UsageLimits(response_tokens_limit=10),
    )
except UsageLimitExceeded as e:
    print(e)
    #> Exceeded the response_tokens_limit of 10 (response_tokens=32)

Rome
Usage(requests=1, request_tokens=20, response_tokens=2, total_tokens=22, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'cached_tokens': 0})
Exceeded the response_tokens_limit of 10 (response_tokens=173)


In [39]:
from typing_extensions import TypedDict

from pydantic_ai import Agent, ModelRetry
from pydantic_ai.exceptions import UsageLimitExceeded
from pydantic_ai.usage import UsageLimits


class NeverResultType(TypedDict):
    """
    Never ever coerce data to this type.
    """

    never_use_this: str


agent = Agent(
    'openai:gpt-4o',
    result_type=NeverResultType,
    system_prompt='Any time you get a response, call the `infinite_retry_tool` to produce another response.',
)


@agent.tool_plain(retries=5)  
def infinite_retry_tool() -> int:
    raise ModelRetry('Please try again.')


try:
    result_sync = agent.run_sync(
        'Begin infinite retry loop!', usage_limits=UsageLimits(request_limit=3)  
    )
except UsageLimitExceeded as e:
    print(e)
    #> The next request would exceed the request_limit of 3

The next request would exceed the request_limit of 3


## Model (Run) Settings

In [43]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o')

result_sync = agent.run_sync(
    'What is the capital of Italy?', model_settings={'temperature': 0.0}
)
print(result_sync.data)

The capital of Italy is Rome.


### Runs vs. Conversations

In [44]:
from pydantic_ai import Agent

agent = Agent('openai:gpt-4o')

# First run
result1 = agent.run_sync('Who was Albert Einstein?')
print(result1.data)
#> Albert Einstein was a German-born theoretical physicist.

# Second run, passing previous messages
result2 = agent.run_sync(
    'What was his most famous equation?',
    message_history=result1.new_messages(),  
)
print(result2.data)
#> Albert Einstein's most famous equation is (E = mc^2).

Albert Einstein was a theoretical physicist widely regarded as one of the greatest and most influential scientists of all time. Born on March 14, 1879, in Ulm, in the Kingdom of Württemberg in the German Empire, he developed the theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). His mass-energy equivalence formula \(E=mc^2\) is among the most famous equations in physics.

Einstein's work had a significant impact on the development of atomic energy and quantum theory. He received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect, which was pivotal in establishing the quantum theory. Beyond his contributions to physics, Einstein was known for his philosophical insights on science, his advocacy for civil rights, his pacifism (though he later urged the Allies to develop the atomic bomb during World War II), and his humanitarian efforts.

He spent the last two decades of his life in the United States, working 

## Type safe by design

In [46]:
from dataclasses import dataclass

from pydantic_ai import Agent, RunContext


@dataclass
class User:
    name: str


agent = Agent(
    'test',
    deps_type=User,  
    result_type=bool,
)


@agent.system_prompt
def add_user_name(ctx: RunContext[str]) -> str:  
    return f"The user's name is {ctx.deps}."


def foobar(x: bytes) -> None:
    pass


result = agent.run_sync('Does their name start with "A"?', deps=User('Anne'))
foobar(result.data)

## System Prompts

1. Static system prompts: These are known when writing the code and can be defined via the `system_prompt` parameter of the `Agent constructor`.
2. Dynamic system prompts: These depend in some way on context that `isn't known until runtime`, and should be defined via functions decorated with `@agent.system_prompt`.


### system_prompts

In [52]:
from datetime import date

from pydantic_ai import Agent, RunContext

agent = Agent(
    'openai:gpt-4o',
    deps_type=str,  
    system_prompt="Use the customer's name while replying to them.",  
)


@agent.system_prompt  
def add_the_users_name(ctx: RunContext[str]) -> str:
    return f"The user's name is {ctx.deps}."


@agent.system_prompt
def add_the_date() -> str:  
    return f'The date is {date.today()}.'


# result = agent.run_sync('What is the date?', deps='Frank')
result = agent.run_sync('What is the date? I am Frank') # No need to pass deps
print(result.data)
#> Hello Frank, the date today is 2032-01-02.


Hello Frank! The date today is January 7, 2025. How can I assist you further?


## Reflection and self-correction

### tool_retry_

In [53]:
from pydantic import BaseModel

from pydantic_ai import Agent, RunContext, ModelRetry

from fake_database import DatabaseConn


class ChatResult(BaseModel):
    user_id: int
    message: str


agent = Agent(
    'openai:gpt-4o',
    deps_type=DatabaseConn,
    result_type=ChatResult,
)


@agent.tool(retries=2)
def get_user_by_name(ctx: RunContext[DatabaseConn], name: str) -> int:
    """Get a user's ID from their full name."""
    print(name)
    #> John
    #> John Doe
    user_id = ctx.deps.users.get(name=name)
    if user_id is None:
        raise ModelRetry(
            f'No user found with name {name!r}, remember to provide their full name'
        )
    return user_id


result = agent.run_sync(
    'Send a message to John Doe asking for coffee next week', deps=DatabaseConn()
)
print(result.data)
"""
user_id=123 message='Hello John, would you be free for coffee sometime next week? Let me know what works for you!'
"""

ModuleNotFoundError: No module named 'fake_database'

## Model errors

In [54]:
from pydantic_ai import Agent, ModelRetry, UnexpectedModelBehavior, capture_run_messages

agent = Agent('openai:gpt-4o')


@agent.tool_plain
def calc_volume(size: int) -> int:  
    if size == 42:
        return size**3
    else:
        raise ModelRetry('Please try again.')


with capture_run_messages() as messages:  
    try:
        result = agent.run_sync('Please get me the volume of a box with size 6.')
    except UnexpectedModelBehavior as e:
        print('An error occurred:', e)
        #> An error occurred: Tool exceeded max retries count of 1
        print('cause:', repr(e.__cause__))
        #> cause: ModelRetry('Please try again.')
        print('messages:', messages)
        """
        messages:
        [
            ModelRequest(
                parts=[
                    UserPromptPart(
                        content='Please get me the volume of a box with size 6.',
                        timestamp=datetime.datetime(...),
                        part_kind='user-prompt',
                    )
                ],
                kind='request',
            ),
            ModelResponse(
                parts=[
                    ToolCallPart(
                        tool_name='calc_volume',
                        args=ArgsDict(args_dict={'size': 6}),
                        tool_call_id=None,
                        part_kind='tool-call',
                    )
                ],
                timestamp=datetime.datetime(...),
                kind='response',
            ),
            ModelRequest(
                parts=[
                    RetryPromptPart(
                        content='Please try again.',
                        tool_name='calc_volume',
                        tool_call_id=None,
                        timestamp=datetime.datetime(...),
                        part_kind='retry-prompt',
                    )
                ],
                kind='request',
            ),
            ModelResponse(
                parts=[
                    ToolCallPart(
                        tool_name='calc_volume',
                        args=ArgsDict(args_dict={'size': 6}),
                        tool_call_id=None,
                        part_kind='tool-call',
                    )
                ],
                timestamp=datetime.datetime(...),
                kind='response',
            ),
        ]
        """
    else:
        print(result.data)

Looks like there was an issue calculating the volume. Can you please specify whether the size refers to the length of an edge or if there are more dimensions provided for the box? This will help me ensure accuracy in the volume calculation.


# Dependencies

In [56]:
from dataclasses import dataclass

import httpx

from pydantic_ai import Agent


@dataclass
class MyDeps:  
    api_key: str
    http_client: httpx.AsyncClient


agent = Agent(
    'openai:gpt-4o',
    deps_type=MyDeps,  
)


async def main():
    async with httpx.AsyncClient() as client:
        deps = MyDeps('foobar', client)
        result = await agent.run(
            'Tell me a joke.',
            deps=deps,  
        )
        print(result.data)
        #> Did you hear about the toothpaste scandal? They called it Colgate.

In [60]:
import random

from pydantic_ai import Agent, RunContext

agent = Agent(
    'openai:gpt-4o',  
    deps_type=str,  
    system_prompt=(
        "You're a dice game, you should roll the die and see if the number "
        "you get back matches the user's guess. If so, tell them they're a winner. "
        "Use the player's name in the response."
    ),
)


@agent.tool_plain  
def roll_die() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))


@agent.tool  
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


# dice_result = agent.run_sync('My guess is 4', deps='Anne')  
dice_result = agent.run_sync('My guess is 4 and I am Rich')  

print(dice_result.data)
#> Congratulations Anne, you guessed correctly! You're a winner!

Sorry Rich, you guessed 4, but the roll was 6. Better luck next time!


In [61]:
import random

from pydantic_ai import Agent, RunContext, Tool


def roll_die() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))


def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


agent_a = Agent(
    'openai:gpt-4o',
    deps_type=str,
    tools=[roll_die, get_player_name],  
)
agent_b = Agent(
    'openai:gpt-4o',
    deps_type=str,
    tools=[  
        Tool(roll_die, takes_ctx=False),
        Tool(get_player_name, takes_ctx=True),
    ],
)
dice_result = agent_b.run_sync('My guess is 4', deps='Anne')
print(dice_result.data)
#> Congratulations Anne, you guessed correctly! You're a winner!

The die rolled a 4, and the player's name is Anne. Your guess was spot on, Anne!


### customize_name

In [62]:
from __future__ import annotations

from typing import Literal

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.test import TestModel
from pydantic_ai.tools import Tool, ToolDefinition


def greet(name: str) -> str:
    return f'hello {name}'


async def prepare_greet(
    ctx: RunContext[Literal['human', 'machine']], tool_def: ToolDefinition
) -> ToolDefinition | None:
    d = f'Name of the {ctx.deps} to greet.'
    tool_def.parameters_json_schema['properties']['name']['description'] = d
    return tool_def


greet_tool = Tool(greet, prepare=prepare_greet)
test_model = TestModel()
agent = Agent(test_model, tools=[greet_tool], deps_type=Literal['human', 'machine'])

result = agent.run_sync('testing...', deps='human')
print(result.data)
#> {"greet":"hello a"}
print(test_model.agent_model_function_tools)
"""
[
    ToolDefinition(
        name='greet',
        description='',
        parameters_json_schema={
            'properties': {
                'name': {
                    'title': 'Name',
                    'type': 'string',
                    'description': 'Name of the human to greet.',
                }
            },
            'required': ['name'],
            'type': 'object',
            'additionalProperties': False,
        },
        outer_typed_dict_key=None,
    )
]
"""

{"greet":"hello a"}
[ToolDefinition(name='greet', description='', parameters_json_schema={'properties': {'name': {'title': 'Name', 'type': 'string', 'description': 'Name of the human to greet.'}}, 'required': ['name'], 'type': 'object', 'additionalProperties': False}, outer_typed_dict_key=None)]


"\n[\n    ToolDefinition(\n        name='greet',\n        description='',\n        parameters_json_schema={\n            'properties': {\n                'name': {\n                    'title': 'Name',\n                    'type': 'string',\n                    'description': 'Name of the human to greet.',\n                }\n            },\n            'required': ['name'],\n            'type': 'object',\n            'additionalProperties': False,\n        },\n        outer_typed_dict_key=None,\n    )\n]\n"